In [8]:
import cv2
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from google.colab.patches import cv2_imshow
# Загрузка предобученной модели
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Переключаем модель в режим оценки

# Определяем трансформации для входного изображения
transform = T.Compose([
    T.ToTensor(),
])

# Функция для обработки видео и сохранения результата
def detect_objects_in_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    # Получаем параметры входного видео
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Создаем объект VideoWriter для записи выходного видео
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Преобразование изображения в тензор
        image = transform(frame).unsqueeze(0)  # Добавляем размер батча

        # Выполнение обнаружения объектов
        with torch.no_grad():
            predictions = model(image)

        # Обработка результатов
        boxes = predictions[0]['boxes']
        scores = predictions[0]['scores']
        labels = predictions[0]['labels']

        # Фильтрация по порогу уверенности (например, 0.5)
        threshold = 0.5
        boxes = boxes[scores > threshold].cpu().numpy()
        labels = labels[scores > threshold].cpu().numpy()

        # Визуализация результатов
        for box, label in zip(boxes, labels):
            x1, y1, x2, y2 = box.astype(int)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'Label: {label}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Записываем обработанный кадр в выходное видео
        out.write(frame)

        # Отображение результата (опционально)
       # cv2_imshow( frame)

        # Выход при нажатии клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Освобождаем ресурсы
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Укажите путь к вашему видеофайлу и выходному файлу
video_path = '/content/1112645_Approach_Winter-clothes_3840x2160 (online-video-cutter.com).mp4'
output_path = 'output_video.mp4'

# Запуск обработки видео
detect_objects_in_video(video_path, output_path)